In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer
from googletrans import Translator
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from tqdm import tqdm
from sklearn.model_selection import KFold
from imblearn.under_sampling import InstanceHardnessThreshold
from sklearn.linear_model import LogisticRegression
import numpy as np
import pandas as pd
import openpyxl
from tqdm import tqdm  
from googletrans import Translator
# Translation
from tqdm import tqdm  
import nltk
#nltk.download('stopwords')
#nltk.download('punkt')
from nltk.corpus import stopwords
from tqdm import tqdm
import nltk
from nltk.tokenize import word_tokenize
import re
from tqdm import tqdm
from nltk.stem import WordNetLemmatizer
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler, EditedNearestNeighbours, OneSidedSelection, InstanceHardnessThreshold
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
import numpy as np
from sklearn.model_selection import KFold
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from bidi.algorithm import get_display
import re
import keras
import openpyxl
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import Counter
from nltk.corpus import stopwords
from googletrans import Translator
from keras.models import Sequential
from tensorflow.keras import Sequential
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
#from keras.utils.np_utils import to_categorical
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import OrdinalEncoder
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Activation
from keras.callbacks import CSVLogger, ModelCheckpoint
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.under_sampling import InstanceHardnessThreshold
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Flatten, BatchNormalization

In [ ]:
# Load Data
wb = openpyxl.load_workbook('E:/Research_Work/Bangla_CyberBulling_2024/Code & dataset/dataset/Final_Dataset_94k_Bangla_CyberBulling_Combine_Dataset_94k.xlsx')
ws = wb['Sheet1']
data_rows = []
for row in ws['A1':'B94001']:
    data_cols = [cell.value for cell in row]
    data_rows.append(data_cols)
df = pd.DataFrame(data_rows)
header = df.iloc[0]
df.columns = [header]
df = df.iloc[1:, :]
class_counts = df['label'].value_counts()
print("Class Counts:\n", class_counts)
all_comments = ' '.join(df['comment'].apply(lambda x: ' '.join(map(str, x))).astype(str))

rgx = r"[\u0980-\u09FF]+"
wordcloud = WordCloud(font_path='E:/Research_Work/Bangla_CyberBulling_2024/Code & dataset/Dataset/Font/CHANO___.ttf',regexp=rgx, width=2000, height=1300, background_color='white').generate(all_comments)

plt.figure(figsize=(30, 15))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()
# Data Exploration
print(df.head())
print(df.iloc[:, -1].value_counts())
print(df.shape)
print(df.isna().sum())

# Handle Missing Values
mode = df.iloc[:, -2].value_counts().index[0]
df.iloc[:, -2].fillna(mode, inplace=True)
print(df.isna().sum())
print(df.head())
# Labeling and Encoding
list(df.iloc[:, -1].value_counts().index)
df['nlabel'] = df.label.replace(['not bully', 'troll', 'sexual', 'religious', 'threat'], [0, 1, 2, 3, 4])
df['noutput'] = df.iloc[:, -1].replace(['bully', 'normal'], [1, 0])
print(df.head())
print(df.head())
translator = Translator()
output = []

for i in tqdm(range(len(df))):
    if df.label.values[i] == 'not bully':
        output.append('Normal')
    else:
        output.append('Bully')

df['output'] = output
df['noutput'] = df.iloc[:, -1].replace(['Bully', 'Normal'], [1, 0])


In [ ]:
df


In [ ]:
import pandas as pd


# Print column names
print("Column Names:", df.columns)

# Calculate Total Comments, Total Words, and Unique Words for 'troll' and 'not bully' classes
class_info = []
for label in ['not bully', 'troll', 'sexual', 'religious', 'threat']:
    class_data = df[df.iloc[:, 1] == label].iloc[:, 0]
    total_comments = len(class_data)
    total_words = ' '.join(class_data).split()
    unique_words = set(total_words)

    class_info.append({
        'Class': label,
        'Total Comments': total_comments,
        'Total Words': len(total_words),
        'Unique Words': len(unique_words)
    })

# Display the results
class_info_df = pd.DataFrame(class_info)
print("\nClass-wise Information:")
print(class_info_df)


In [ ]:
df.head()

In [ ]:
print(stopwords.fileids())
sw = stopwords.words('bengali')

new_stopwords = stopwords.words('english')
new_stopwords.append('SampleWord')

df.to_csv('df_english1000.csv')

#nltk.download('wordnet')
#nltk.download('omw-1.4')

wordnet_lemmatizer = WordNetLemmatizer()

M = len(df)
for k in tqdm(range(M)):
    text = df.iloc[:, 0].values[k]
    txt = re.sub(r'http\S+', '', text)
    txt = re.sub(r'[!@#$%^&*?><,./\-+`~|:);(❤}{]', '', txt)

    text_tokens = word_tokenize(txt)

    def detect_benglish(text_tokens):
        benglish = False
        for j in range(len(text_tokens)):
            for i in range(len(text_tokens[j])):
                detector = Translator()
                txt = text_tokens[j]
                if i < len(txt) - 1:
                    if detector.detect(txt[i]).lang == 'bn' and detector.detect(txt[i + 1]).lang == 'en' and txt[
                        i].isalpha() == True and txt[i + 1].isalpha() == True:
                        benglish = True
        return benglish

    def handle_benglish(text_tokens):
        for j in range(len(text_tokens)):
            s = []
            for i in range(len(text_tokens[j])):
                detector = Translator()
                txt = text_tokens[j]
                if i < len(txt) - 1:
                    if detector.detect(txt[i]).lang == 'bn' and detector.detect(txt[i + 1]).lang == 'en' and txt[
                        i].isalpha() == True and txt[i + 1].isalpha() == True:
                        s.append(i)
                    if txt[i] == '|':
                        s.append(i)
            for i in range(len(s)):
                txt = txt.replace(txt[s[i]], txt[s[i]] + " ")
            text_tokens[j] = txt
        tstring = str(' '.join(text_tokens)).lower()
        text_tokens = word_tokenize(tstring)
        return text_tokens

    remove_sw = [word for word in text_tokens if not word in sw]
    un_items = np.unique(remove_sw)
    r_sw = [wordnet_lemmatizer.lemmatize(w) for w in un_items]
    bn_tokens = []

    def tanslate_bengali(r_sw):
        for i in range(len(r_sw)):
            bn_tokens.append(translator.translate(r_sw[i], dest='bn').text)
        return bn_tokens

    bn_token = r_sw
    df.iloc[:, 0].values[k] = ' '.join(bn_token)

df.iloc[:, -1].value_counts()
df.iloc[:, 0]

df[:].iloc[:, 0].shape


In [ ]:
df

In [ ]:
training_sentences = []
train_sentences=df['comment'].values
train_labels=df['nlabel'].values
for i in range(train_sentences.shape[0]): 
    #print(train_sentences[i])
    x=str(train_sentences[i])
    training_sentences.append(x)
    
training_sentences=np.array(training_sentences)

train_labels=keras.utils.to_categorical(train_labels)

print("training_sentences shape: "+str(training_sentences.shape))
print("train_labels shape: "+str(train_labels.shape))

In [ ]:
print(training_sentences[1])
print(train_labels[0])

In [ ]:
vocab_size = 20000
embedding_dim = 300
max_length = 100
trunc_type='post'
oov_tok = "<OOV>"
print(training_sentences.shape)
print(train_labels.shape)

In [ ]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
print(len(word_index))
print("Word index length:"+str(len(tokenizer.word_index)))
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

In [ ]:
print("Sentence :--> \n")
print(training_sentences[15]+"\n")
print("Sentence Tokenized and Converted into Sequence :--> \n")
print(str(sequences[15])+"\n")
print("After Padding the Sequence with padding length 100 :--> \n")
print(padded[15])
print("Padded shape(training): "+str(padded.shape))

In [ ]:
training_sentences

In [ ]:
# Imbalanced Data Handling
cc = InstanceHardnessThreshold(random_state=10, estimator=LogisticRegression())
X, y = cc.fit_resample(padded, train_labels)
X.shape, y.shape

In [ ]:
# Cross-Validation
kf = KFold(n_splits=5, random_state=10, shuffle=True)
for train_index, test_index in kf.split(X, y):
    padded, train_labels = X[train_index], X[test_index]
    testing_padded, testing_padded1 = y[train_index], y[test_index]
print('complete')
print('train:', padded.shape, train_labels.shape, 'test:', testing_padded.shape, testing_padded1.shape)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:

# Create a Sequential model
cnn_model = Sequential()

# Add an Embedding layer
cnn_model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))

# Add a 1D Convolutional layer
cnn_model.add(Conv1D(filters=200, kernel_size=3, activation='relu'))

# Add a Global Max Pooling layer to reduce dimensionality
cnn_model.add(GlobalMaxPooling1D())

# Add one or more Dense layers for classification
cnn_model.add(Dense(50, activation='relu'))
cnn_model.add(Dense(5, activation='softmax'))  # Assuming binary classification

# Compile the model
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Display the model summary
cnn_model.summary()

# Assuming you've trained the model using the provided code
history = cnn_model.fit(padded, testing_padded, epochs=15, batch_size=32, validation_data=(train_labels, testing_padded1), use_multiprocessing=True, workers=8)

loss_and_metrics = cnn_model.evaluate(train_labels, testing_padded1, batch_size=32)
accuracy = loss_and_metrics[1]

# Predict on test data
y_pred = cnn_model.predict(train_labels)
y_pred_classes = tf.argmax(y_pred, axis=1)

# Convert one-hot encoded labels to integers
y_true = tf.argmax(testing_padded1, axis=1)

# Calculate Precision, Recall, and F1 Score
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')

# Print metrics
print(f"CNN Accuracy: {accuracy*100:.2f}%")
print(f"Precision: {precision*100:.2f}%")
print(f"Recall: {recall*100:.2f}%")
print(f"F1 Score: {f1*100:.2f}%")

In [ ]:

print(history.history.keys())
loss = history.history['loss']
val_loss = history.history['val_loss']
plt.plot(loss)
plt.plot(val_loss)
plt.title('CNN model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['loss', 'val_loss'])
plt.show()

accuracy = history.history['accuracy']
val_accuracy= history.history['val_accuracy']
plt.plot(accuracy)
plt.plot(val_accuracy)
plt.title('CNN model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['accuracy', 'val_accuracy'])
plt.show()

# Assuming you have already trained the model and obtained predictions
predictions = cnn_model.predict(train_labels)
predicted_labels = np.argmax(predictions, axis=1)
actual_labels = np.argmax(testing_padded1, axis=1)

# Create confusion matrix
conf_matrix = confusion_matrix(actual_labels, predicted_labels)

# Define class names
class_names = ['not bully', 'troll', 'sexual', 'religious', 'threat']

# Plot colorful confusion matrix
plt.figure(figsize=(6, 5))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
 # Display values on the plot without box and color
for i in range(len(class_names)):
    for j in range(len(class_names)):
        plt.text(j + 0.5, i + 0.5, f'{conf_matrix[i, j]}', ha='center', va='center', color='black', fontsize=10)
# Set labels and title
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix CNN")
# Display the plot
plt.show()

# ... (Previous code remains unchanged)

# Predict probabilities for each class
y_probs = cnn_model.predict(train_labels)

# Convert one-hot encoded labels to integers
y_true = np.argmax(testing_padded1, axis=1)

class_names = ['Not Bully', 'Troll', 'Sexual', 'Religious', 'Threat']
# Plot ROC curve for each class
plt.figure(figsize=(6, 4))

for i in range(len(class_names)):
    # Compute ROC curve and ROC area for each class
    fpr, tpr, _ = roc_curve((y_true == i).astype(int), y_probs[:, i])
    roc_auc = auc(fpr, tpr)

    # Plot ROC curve with class names
    plt.plot(fpr, tpr, label=f'{class_names[i]} (AUC = {roc_auc:.2f})')

# Plot diagonal line for reference
plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Random')

# Customize the plot
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Multi-Class Classification CNN')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Dense, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical


# Model Definition - CLSTM
clstm_model = Sequential()
clstm_model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
clstm_model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
clstm_model.add(MaxPooling1D(pool_size=2))
clstm_model.add(LSTM(100, return_sequences=True))
clstm_model.add(LSTM(100))
clstm_model.add(Dense(50, activation='relu'))
clstm_model.add(Dense(5, activation='softmax'))

# Compile the model
adam = Adam(learning_rate=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False)
clstm_model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

# Display the model summary
clstm_model.summary()

# Training
history = clstm_model.fit(padded, testing_padded, epochs=15, batch_size=32, validation_data=(train_labels, testing_padded1), use_multiprocessing=True, workers=8)

# Evaluate Test Accuracy
loss_and_metrics = clstm_model.evaluate(train_labels, testing_padded1, batch_size=32)
accuracy = loss_and_metrics[1]

# Predict on test data
y_pred = clstm_model.predict(train_labels)
y_pred_classes = tf.argmax(y_pred, axis=1)

# Convert one-hot encoded labels to integers
y_true = tf.argmax(testing_padded1, axis=1)

# Calculate Precision, Recall, and F1 Score
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')

# Print metrics
print(f"Accuracy: {accuracy*100:.2f}%")
print(f"Precision: {precision*100:.2f}%")
print(f"Recall: {recall*100:.2f}%")
print(f"F1 Score: {f1*100:.2f}%")

In [ ]:
print(history.history.keys())
loss = history.history['loss']
val_loss = history.history['val_loss']
plt.plot(loss)
plt.plot(val_loss)
plt.title('CLSTM model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['loss', 'val_loss'])
plt.show()

accuracy = history.history['accuracy']
val_accuracy= history.history['val_accuracy']
plt.plot(accuracy)
plt.plot(val_accuracy)
plt.title('CLSTM model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['accuracy', 'val_accuracy'])
plt.show()


import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

# Assuming you have already trained the model and obtained predictions
predictions = clstm_model.predict(train_labels)
predicted_labels = np.argmax(predictions, axis=1)
actual_labels = np.argmax(testing_padded1, axis=1)

# Create confusion matrix
conf_matrix = confusion_matrix(actual_labels, predicted_labels)

# Define class names
class_names = ['not bully', 'troll', 'sexual', 'religious', 'threat']

# Plot colorful confusion matrix
plt.figure(figsize=(6, 5))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
 # Display values on the plot without box and color
for i in range(len(class_names)):
    for j in range(len(class_names)):
        plt.text(j + 0.5, i + 0.5, f'{conf_matrix[i, j]}', ha='center', va='center', color='black', fontsize=10)
# Set labels and title
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix CLSTM")
# Display the plot
plt.show()

import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import numpy as np

# ... (Previous code remains unchanged)

# Predict probabilities for each class
y_probs = clstm_model.predict(train_labels)

# Convert one-hot encoded labels to integers
y_true = np.argmax(testing_padded1, axis=1)

class_names = ['Not Bully', 'Troll', 'Sexual', 'Religious', 'Threat']
# Plot ROC curve for each class
plt.figure(figsize=(6, 4))

for i in range(len(class_names)):
    # Compute ROC curve and ROC area for each class
    fpr, tpr, _ = roc_curve((y_true == i).astype(int), y_probs[:, i])
    roc_auc = auc(fpr, tpr)

    # Plot ROC curve with class names
    plt.plot(fpr, tpr, label=f'{class_names[i]} (AUC = {roc_auc:.2f})')

# Plot diagonal line for reference
plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Random')

# Customize the plot
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Multi-Class Classification CLSTM')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Model Definition - RNN
rnn_model = Sequential()
rnn_model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
rnn_model.add(SimpleRNN(100, return_sequences=True))  # Use LSTM if needed for longer-term dependencies
rnn_model.add(SimpleRNN(100))
rnn_model.add(Dense(50, activation='relu'))
rnn_model.add(Dense(5, activation='softmax'))

# Compile the model
adam = Adam(learning_rate=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False)
rnn_model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

# Display the model summary
rnn_model.summary()

# Training
history = rnn_model.fit(padded, testing_padded, epochs=15, batch_size=32, validation_data=(train_labels, testing_padded1), use_multiprocessing=True, workers=8)

# Evaluate Test Accuracy
loss_and_metrics = rnn_model.evaluate(train_labels, testing_padded1, batch_size=32)
accuracy = loss_and_metrics[1]

# Predict on test data
y_pred = rnn_model.predict(train_labels)
y_pred_classes = tf.argmax(y_pred, axis=1)

# Convert one-hot encoded labels to integers
y_true = tf.argmax(testing_padded1, axis=1)

# Calculate Precision, Recall, and F1 Score
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')

# Print metrics
print(f"Accuracy: {accuracy*100:.2f}%")
print(f"Precision: {precision*100:.2f}%")
print(f"Recall: {recall*100:.2f}%")
print(f"F1 Score: {f1*100:.2f}%")

In [ ]:
print(history.history.keys())
loss = history.history['loss']
val_loss = history.history['val_loss']
plt.plot(loss)
plt.plot(val_loss)
plt.title('RNN model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['loss', 'val_loss'])
plt.show()

accuracy = history.history['accuracy']
val_accuracy= history.history['val_accuracy']
plt.plot(accuracy)
plt.plot(val_accuracy)
plt.title('RNN model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['accuracy', 'val_accuracy'])
plt.show()

# Assuming you have already trained the model and obtained predictions
predictions = rnn_model.predict(train_labels)
predicted_labels = np.argmax(predictions, axis=1)
actual_labels = np.argmax(testing_padded1, axis=1)

# Create confusion matrix
conf_matrix = confusion_matrix(actual_labels, predicted_labels)

# Define class names
class_names = ['not bully', 'troll', 'sexual', 'religious', 'threat']

# Plot colorful confusion matrix
plt.figure(figsize=(6, 5))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
 # Display values on the plot without box and color
for i in range(len(class_names)):
    for j in range(len(class_names)):
        plt.text(j + 0.5, i + 0.5, f'{conf_matrix[i, j]}', ha='center', va='center', color='black', fontsize=10)
# Set labels and title
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix RNN")
# Display the plot
plt.show()

# Predict probabilities for each class
y_probs = rnn_model.predict(train_labels)

# Convert one-hot encoded labels to integers
y_true = np.argmax(testing_padded1, axis=1)

class_names = ['Not Bully', 'Troll', 'Sexual', 'Religious', 'Threat']
# Plot ROC curve for each class
plt.figure(figsize=(6, 4))

for i in range(len(class_names)):
    # Compute ROC curve and ROC area for each class
    fpr, tpr, _ = roc_curve((y_true == i).astype(int), y_probs[:, i])
    roc_auc = auc(fpr, tpr)

    # Plot ROC curve with class names
    plt.plot(fpr, tpr, label=f'{class_names[i]} (AUC = {roc_auc:.2f})')

# Plot diagonal line for reference
plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Random')

# Customize the plot
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Multi-Class Classification RNN')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, GRU, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Model Definition - BiGRU
bigru_model = Sequential()
bigru_model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
bigru_model.add(Bidirectional(GRU(100, return_sequences=True)))
bigru_model.add(Bidirectional(GRU(100)))
bigru_model.add(Dense(50, activation='relu'))
bigru_model.add(Dense(5, activation='softmax'))

# Compile the model
adam = Adam(learning_rate=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False)
bigru_model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

# Display the model summary
bigru_model.summary()

# Training
history = bigru_model.fit(padded, testing_padded, epochs=15, batch_size=32, validation_data=(train_labels, testing_padded1), use_multiprocessing=True, workers=8)

# Evaluate Test Accuracy
loss_and_metrics = bigru_model.evaluate(train_labels, testing_padded1, batch_size=32)
accuracy = loss_and_metrics[1]

# Predict on test data
y_pred = bigru_model.predict(train_labels)
y_pred_classes = tf.argmax(y_pred, axis=1)

# Convert one-hot encoded labels to integers
y_true = tf.argmax(testing_padded1, axis=1)

# Calculate Precision, Recall, and F1 Score
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')

# Print metrics
print(f"Accuracy: {accuracy*100:.2f}%")
print(f"Precision: {precision*100:.2f}%")
print(f"Recall: {recall*100:.2f}%")
print(f"F1 Score: {f1*100:.2f}%")

In [ ]:
print(history.history.keys())
loss = history.history['loss']
val_loss = history.history['val_loss']
plt.plot(loss)
plt.plot(val_loss)
plt.title('BiGRU model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['loss', 'val_loss'])
plt.show()

accuracy = history.history['accuracy']
val_accuracy= history.history['val_accuracy']
plt.plot(accuracy)
plt.plot(val_accuracy)
plt.title('BiGRU model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['accuracy', 'val_accuracy'])
plt.show()


import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

# Assuming you have already trained the model and obtained predictions
predictions = bigru_model.predict(train_labels)
predicted_labels = np.argmax(predictions, axis=1)
actual_labels = np.argmax(testing_padded1, axis=1)

# Create confusion matrix
conf_matrix = confusion_matrix(actual_labels, predicted_labels)

# Define class names
class_names = ['not bully', 'troll', 'sexual', 'religious', 'threat']

# Plot colorful confusion matrix
plt.figure(figsize=(6, 5))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
 # Display values on the plot without box and color
for i in range(len(class_names)):
    for j in range(len(class_names)):
        plt.text(j + 0.5, i + 0.5, f'{conf_matrix[i, j]}', ha='center', va='center', color='black', fontsize=10)
# Set labels and title
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix BiGRU")
# Display the plot
plt.show()

import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import numpy as np

# ... (Previous code remains unchanged)

# Predict probabilities for each class
y_probs = bigru_model.predict(train_labels)

# Convert one-hot encoded labels to integers
y_true = np.argmax(testing_padded1, axis=1)

class_names = ['Not Bully', 'Troll', 'Sexual', 'Religious', 'Threat']
# Plot ROC curve for each class
plt.figure(figsize=(6, 4))

for i in range(len(class_names)):
    # Compute ROC curve and ROC area for each class
    fpr, tpr, _ = roc_curve((y_true == i).astype(int), y_probs[:, i])
    roc_auc = auc(fpr, tpr)

    # Plot ROC curve with class names
    plt.plot(fpr, tpr, label=f'{class_names[i]} (AUC = {roc_auc:.2f})')

# Plot diagonal line for reference
plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Random')

# Customize the plot
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Multi-Class Classification BiGRU')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Model Definition - DNN
dnn_model = Sequential()
dnn_model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
dnn_model.add(Flatten())
dnn_model.add(Dense(256, activation='relu'))
dnn_model.add(Dense(128, activation='relu'))
dnn_model.add(Dense(64, activation='relu'))
dnn_model.add(Dense(5, activation='softmax'))

# Compile the model
adam = Adam(learning_rate=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False)
dnn_model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

# Display the model summary
dnn_model.summary()

# Training
history = dnn_model.fit(padded, testing_padded, epochs=15, batch_size=32, validation_data=(train_labels, testing_padded1), use_multiprocessing=True, workers=8)

# Evaluate Test Accuracy
loss_and_metrics = dnn_model.evaluate(train_labels, testing_padded1, batch_size=32)
accuracy = loss_and_metrics[1]

# Predict on test data
y_pred = dnn_model.predict(train_labels)
y_pred_classes = tf.argmax(y_pred, axis=1)

# Convert one-hot encoded labels to integers
y_true = tf.argmax(testing_padded1, axis=1)

# Calculate Precision, Recall, and F1 Score
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')

# Print metrics
print(f"Accuracy: {accuracy*100:.2f}%")
print(f"Precision: {precision*100:.2f}%")
print(f"Recall: {recall*100:.2f}%")
print(f"F1 Score: {f1*100:.2f}%")


In [ ]:
print(history.history.keys())
loss = history.history['loss']
val_loss = history.history['val_loss']
plt.plot(loss)
plt.plot(val_loss)
plt.title('DNN model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['loss', 'val_loss'])
plt.show()

accuracy = history.history['accuracy']
val_accuracy= history.history['val_accuracy']
plt.plot(accuracy)
plt.plot(val_accuracy)
plt.title('DNN model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['accuracy', 'val_accuracy'])
plt.show()


import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

# Assuming you have already trained the model and obtained predictions
predictions = dnn_model.predict(train_labels)
predicted_labels = np.argmax(predictions, axis=1)
actual_labels = np.argmax(testing_padded1, axis=1)

# Create confusion matrix
conf_matrix = confusion_matrix(actual_labels, predicted_labels)

# Define class names
class_names = ['not bully', 'troll', 'sexual', 'religious', 'threat']

# Plot colorful confusion matrix
plt.figure(figsize=(6, 5))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
 # Display values on the plot without box and color
for i in range(len(class_names)):
    for j in range(len(class_names)):
        plt.text(j + 0.5, i + 0.5, f'{conf_matrix[i, j]}', ha='center', va='center', color='black', fontsize=10)
# Set labels and title
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix DNN")
# Display the plot
plt.show()

import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import numpy as np

# ... (Previous code remains unchanged)

# Predict probabilities for each class
y_probs = dnn_model.predict(train_labels)

# Convert one-hot encoded labels to integers
y_true = np.argmax(testing_padded1, axis=1)

class_names = ['Not Bully', 'Troll', 'Sexual', 'Religious', 'Threat']
# Plot ROC curve for each class
plt.figure(figsize=(6, 4))

for i in range(len(class_names)):
    # Compute ROC curve and ROC area for each class
    fpr, tpr, _ = roc_curve((y_true == i).astype(int), y_probs[:, i])
    roc_auc = auc(fpr, tpr)

    # Plot ROC curve with class names
    plt.plot(fpr, tpr, label=f'{class_names[i]} (AUC = {roc_auc:.2f})')

# Plot diagonal line for reference
plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Random')

# Customize the plot
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Multi-Class Classification DNN')
plt.legend()
plt.grid(True)
plt.show()
